# PRM IMPLEMENTATION FOR FINDING VARIOUS PATHS THEN USING A* TO FIND SHORTEST PATHS FROM VARIOUS POINTS

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from queue import PriorityQueue
import random




# Function for identifying the obstacles in the maze.
def obstacle_identification(image):
    
    image = cv.imread(image, cv.IMREAD_GRAYSCALE) # Here image is read as numpy array using cv.imread and converted to grayscale image.
    _, thresholded = cv.threshold(image, 1, 255, cv.THRESH_BINARY) # Used threshold to convert grayscale to black(0) and white(255).
    # Obstacles represented by black and free space by white.
    return thresholded

obstacle_map = obstacle_identification('maze.png')





#Function for Valid sample space for nodes generation.
def sample_free_space(obstacle_map, num_samples, edge_distance=20):
    free_space = np.argwhere(obstacle_map == 255) # Free space represented by White.(This would contain all the indices of the points in the image where White colour is present.(Free space))
    valid_indices = []
    for idx in free_space:
        if (
            idx[0] > edge_distance
            and idx[0] < obstacle_map.shape[0] - edge_distance
            and idx[1] > edge_distance
            and idx[1] < obstacle_map.shape[1] - edge_distance
        ):
            valid_indices.append(idx)
    valid_indices = np.array(valid_indices)
    indices = np.random.choice(len(valid_indices), num_samples, replace=False) # This will create a numpy array of from 0 to the length of thee valid indices (exclusive).
    # Then it will randomly choose the indices from the array and the no would be given no of nodes. 
    return valid_indices[indices]



# Function defined for connecting the nodes.
def connect_nodes(nodes, obstacle_map, num_neighbors):  # Here nodes will be all the indices found after getting them from the above function.
    kdtree = KDTree(nodes)
    edges = {}
    for i, node in enumerate(nodes):
        distances, indices = kdtree.query(node, k=num_neighbors+1)
        ''' Here i will be returned the distances and the indices of the nodes
        which are nearest to my current node. Here k is set as no of neighbours+1 because 1st neighbour will be the node itself
        as well as the first element of the distance in each case would be zero only.'''
        
        connections = []  # Made an empty list for storing connections inside the for loop. 
        for j in indices[1:]:
            if not collision(node, nodes[j], obstacle_map):
                connections.append(j)    # Making the connections by checkig that there is not any obstacle between the nodes.
        edges[i] = connections           # Storing the connections in the dictionary edges.
    return edges




# Function for checking that is there any collision between any two nodes.
def collision(node1, node2, obstacle_map):  
    line = np.linspace(node1, node2, num=100) # Using np.linspace i have divided both the co-ordinates of the point in several no of points.
    
    for point in line:
        if obstacle_map[int(round(point[0])), int(round(point[1]))] == 0: # Using loop I have checked that is there any black point in between the nodes or not.
            return True # Returned True if there is an obstacle in between the nodes, else False.  
    return False



def find_nearest_node(nodes, coordinate):
    distances = np.linalg.norm(nodes - coordinate, axis=1)
    nearest_node_index = np.argmin(distances)
    return nearest_node_index





def heuristic(node1, node2):    
    return np.linalg.norm(node1 - node2)



def astar(graph, nodes, start, goal):  # Here graph represents the array containing the indices which can be the possible connections at each node.
    
    frontier = PriorityQueue()   # Here i have defined a priority queue which will give priority to the node with less priority value
    # So i will decide the priority value based on the cost value at each of the node.

    frontier.put((0, start))  # The starting node is added to the queue with a priority of 0.
    
    came_from = {}            # Used for storing the index of the parent node at each node. 
    cost_so_far = {}          # Used for storing cost for each node.
    came_from[start] = None   # No parent node for start value
    cost_so_far[start] = 0    # Intialized cost for starting node = 0
    
    
    
    while not frontier.empty():
        _, current = frontier.get() # Using this i will get the node with maximum priority means with the minimum cost. 

        if current == goal:
            break

        for next_node in graph[current]:  
            new_cost = cost_so_far[current] + heuristic(nodes[current], nodes[next_node]) # This is the cost which is the defined as cost so far and the distance between the nodes.
            if next_node not in cost_so_far :
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(nodes[next_node], nodes[goal]) 
                frontier.put((priority, next_node))
                came_from[next_node] = current

                
    path = []                              # This is the list created for storing the path found by A* algorithm. 
    current = goal
    while current != start:
        path.append(current)
        if current not in came_from:
            print("No path found from start to goal")
            print("Try again")
            return []
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path



def visualize(obstacle_map, nodes, edges, path):
    # Display the obstacle map
    plt.imshow(obstacle_map, cmap='gray')

    # Plot nodes on the obstacle map
    plt.scatter(nodes[:, 1], nodes[:, 0], color='red', marker='*', label='Nodes')

    # Plot edges between connected nodes
    for i, connections in edges.items():
        for j in connections:
            node1 = nodes[i]
            node2 = nodes[j]
            # Check if there's no collision between nodes
            if not collision(node1, node2, obstacle_map):
                # Plot edge between nodes
                plt.plot([node1[1], node2[1]], [node1[0], node2[0]], color=(0.5, 0.5, 1.0), linewidth=0.5)

    # Plot the shortest path, if available
    if path:
        # Extract coordinates of nodes in the shortest path
        path_nodes = np.array([nodes[node] for node in path])
        # Plot the path
        plt.plot(path_nodes[:, 1], path_nodes[:, 0], color='blue', linewidth=2, label='Shortest Path')

    # Add legend, title, and display the plot
    plt.legend()
    plt.title('PRM Roadmap with A* Shortest Path')
    plt.show()


    

def prm_algorithm(obstacle_map, num_samples, num_neighbors):
    # For both the visulizations i have made the different 
    a=str(input("Tell that 'EASY' or 'HARD': "))
    if a=="EASY":
        
        
        im=cv.imread('maze.png')
        Width= im.shape[1]
        Height= im.shape[0]
        START_EASY = np.array([Height,round((Width/7)*0.5)]) # These i have defined by visullization from the image given.
        END_EASY = np.array([Height,round(2*(Width/7)*0.5)])
    
        nodes = sample_free_space(obstacle_map, num_samples)
        edges = connect_nodes(nodes, obstacle_map, num_neighbors)
    
    
        start_index = find_nearest_node(nodes,START_EASY ) 
        end_index = find_nearest_node(nodes,END_EASY )
        
        start = start_index
        goal = end_index
    
        path = astar(edges, nodes, start, goal)
        visualize(obstacle_map, nodes, edges, path)
        
    else:
        
        im=cv.imread('maze.png')
        Width= im.shape[1]
        Height= im.shape[0]
        START_HARD = np.array([0,round(3*(Width/7)*0.5)]) # These i have defined by visullization from the image given.
        END_HARD = np.array([Height,Width])                 
    
        nodes = sample_free_space(obstacle_map, num_samples)
        edges = connect_nodes(nodes, obstacle_map, num_neighbors)
    
    
        start_index = find_nearest_node(nodes,START_HARD )  # Found nearest node to the starting and the ending point.
        end_index = find_nearest_node(nodes,END_HARD )        
        
        start = start_index
        goal = end_index
    
        path = astar(edges, nodes, start, goal)
        visualize(obstacle_map, nodes, edges, path)

        
        


def main():
    num_samples = 200   # These are the no of nodes to be generated
    num_neighbors = 50  # These are the the no of max neighbours for each node
    prm_algorithm(obstacle_map, num_samples, num_neighbors)

if __name__ == "__main__":
    main()

# TASK_02 PART 2

In [ ]:
import cv2 as cv
import numpy as np
import pygame
from scipy.spatial import KDTree
from queue import PriorityQueue
import random
import time
import matplotlib.pyplot as plt

# Initialize Pygame for displaying graphical elements
pygame.init()
WINDOW_WIDTH = 500
WINDOW_HEIGHT = 500
ACTIVE_WINDOW = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

def obstacle_identification(grid):
    """
    Convert the grid to a binary map where obstacles are 0 and free space is 255.
    """
    obs_map = np.where(grid == 0, 0, 255).astype(np.uint8)
    return obs_map


def sample_free_space(obstacle_map, num_samples, edge_distance=20):
    """
    Randomly sample a specified number of points from the free space in the obstacle map, ensuring a 
    minimum distance from the edges of the map.

    Parameters:
    - obstacle_map: numpy array representing the obstacle space (0 = obstacle, 255 = free space).
    - num_samples: number of points to sample.
    - edge_distance: minimum distance from the edges of the map for sampled points.

    Returns:
    - sampled_points: array of sampled points in free space away from edges.
    """
    # Retrieve indices where free space is present
    free_space = np.argwhere(obstacle_map == 255)

    # Filter out points close to the edges of the map
    valid_indices = [
        idx for idx in free_space
        if idx[0] > edge_distance and
           idx[0] < obstacle_map.shape[0] - edge_distance and
           idx[1] > edge_distance and
           idx[1] < obstacle_map.shape[1] - edge_distance
    ]

    # Convert list to numpy array for easier handling
    valid_indices = np.array(valid_indices)

    # Randomly select specified number of indices from the valid indices
    indices = np.random.choice(len(valid_indices), num_samples, replace=False)
    sampled_points = valid_indices[indices]
    
    return sampled_points


def connect_nodes(nodes, obstacle_map, num_neighbors):
    """
    Connect nodes using a KDTree to efficiently find nearest neighbors, while ensuring no connection
    crosses an obstacle.
    """
    kdtree = KDTree(nodes)
    edges = {}
    for i, node in enumerate(nodes):
        distances, indices = kdtree.query(node, k=num_neighbors+1)
        connections = []
        for j in indices[1:]:  # Skip the first index because it is the node itself
            if not collision(node, nodes[j], obstacle_map):
                connections.append(j)
        edges[i] = connections
    return edges

def collision(node1, node2, obstacle_map):
    """
    Check if a straight line between two nodes intersects any obstacles.
    """
    line = np.linspace(node1, node2, num=100)  # Generates points along the line
    for point in line:
        if obstacle_map[int(round(point[0])), int(round(point[1]))] == 0:
            return True
    return False

def find_nearest_node(nodes, coordinate):
    """
    Find the nearest node in the nodes array to a given coordinate.
    """
    distances = np.linalg.norm(nodes - coordinate, axis=1)
    nearest_node_index = np.argmin(distances)
    return nearest_node_index

def heuristic(node1, node2):
    """
    Heuristic function for A* pathfinding, using Euclidean distance.
    """
    return np.linalg.norm(node1 - node2)

def astar(graph, nodes, start, goal):
    """
    A* algorithm implementation to find the shortest path from start to goal node in a graph.
    """
    frontier = PriorityQueue()
    frontier.put((0, start))
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        _, current = frontier.get()
        if current == goal:
            break
        for next_node in graph[current]:
            new_cost = cost_so_far[current] + heuristic(nodes[current], nodes[next_node])
            if next_node not in cost_so_far:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(nodes[next_node], nodes[goal]) 
                frontier.put((priority, next_node))
                came_from[next_node] = current

    path = []
    current = goal
    while current != start:
        path.append(current)
        if current not in came_from:
            print("No path found from start to goal")
            print("Try again")
            return []
        current = came_from[current]
    path.append(start)
    path.reverse()
    intermediate_path = []
    for i in range(len(path) - 1):
        start_node = nodes[path[i]]
        end_node = nodes[path[i+1]]
        line = np.linspace(start_node, end_node, num=100)
        for point in line:
            intermediate_path.append((int(round(point[0])), int(round(point[1]))))
    
    return intermediate_path

def custom_planner(grid, matrix, start_loc, goal_loc):
    """
    Custom path planning function integrating obstacle detection, node sampling, and A* pathfinding.
    """
    start_time = time.time()
    grid_array = np.array(grid)
    grid_array = grid_array[:, :, 0]
    
    obstacle_map = obstacle_identification(grid_array) 
    num_samples = 100
    num_neighbors = 20
    nodes = sample_free_space(obstacle_map, num_samples)
    
    # Ensure start and goal are included in the nodes
    if tuple(start_loc) not in nodes:
        nodes = np.vstack([nodes, start_loc])
    if tuple(goal_loc) not in nodes:
        nodes = np.vstack([nodes, goal_loc])
    
    edges = connect_nodes(nodes, obstacle_map, num_neighbors)
    
    # Find nearest nodes to start and goal
    start = find_nearest_node(nodes, start_loc)
    goal = find_nearest_node(nodes, goal_loc)
    
    # Execute A* algorithm
    path = astar(edges, nodes, start, goal)
    end_time = time.time()
    runtime = (end_time - start_time)
    return (path, runtime)

# Integration with an external simulation environment
from autonavsim2d.autonavsim2d import AutoNavSim2D

nav = AutoNavSim2D(
    custom_planner=custom_planner,
    custom_motion_planner='default',
    window='amr'
)
nav.run()  # Run the navigation simulation
